In [11]:
!pip install tensorflow

In [1]:
from scipy.io import arff
import pandas as pd

data, meta = arff.loadarff("mnist_784.arff")
df = pd.DataFrame(data)
print(df.head())

   pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   pixel10  ...  pixel776  pixel777  pixel778  pixel779  pixel780  pixel781  \
0      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
1      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
2      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
3      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
4      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   

   pixel782  pixel783  pixel784  class  
0       0.0       0.0

In [4]:
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import joblib

# 1. ARFF 파일 로드 (같은 경로에 mnist_784.arff 파일이 있어야 함)
data, meta = arff.loadarff("mnist_784.arff")
df = pd.DataFrame(data)

# 2. 클래스 byte → 문자열 변환
df['class'] = df['class'].str.decode('utf-8')

# 3. 특징과 라벨 분리
X = df.drop(columns=['class']).values
y = df['class'].astype(int).values

# 4. 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. 모델 학습
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

# 6. 저장
joblib.dump(model, "mnist_knn.joblib")
print("✅ 모델이 저장되었습니다: mnist_knn.joblib")

✅ 모델이 저장되었습니다: mnist_knn.joblib


In [1]:
# 숫자 인식으로 회전 각도 입력하는 통합 이미지 보정기 (필터 중복 제거 및 누적 적용 지원)

import sys
import cv2
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QPushButton, QFileDialog,
    QVBoxLayout, QWidget, QComboBox, QHBoxLayout, QSlider, QDialog, QLineEdit, QMessageBox
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
from PyQt5.QtCore import Qt, QPoint
import joblib
import os

class BeforeAfterDialog(QDialog):
    def __init__(self, original, processed):
        super().__init__()
        self.setWindowTitle("Before / After 비교")
        self.setFixedSize(1100, 550)

        layout = QHBoxLayout()
        before_label = QLabel("Before")
        before_pixmap = self.cvToQPixmap(original)
        before_label.setPixmap(before_pixmap)
        before_label.setFixedSize(512, 512)

        after_label = QLabel("After")
        after_pixmap = self.cvToQPixmap(processed)
        after_label.setPixmap(after_pixmap)
        after_label.setFixedSize(512, 512)

        layout.addWidget(before_label)
        layout.addWidget(after_label)
        self.setLayout(layout)

    def cvToQPixmap(self, img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, ch = img.shape
        qimg = QImage(img.data, w, h, ch * w, QImage.Format_RGB888)
        return QPixmap.fromImage(qimg).scaled(512, 512, Qt.KeepAspectRatio)

class DrawDigitDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("회전 각도 숫자 입력")
        self.setFixedSize(280, 280)
        self.image = QImage(self.size(), QImage.Format_RGB32)
        self.image.fill(Qt.white)
        self.drawing = False
        self.lastPoint = QPoint()
        self.model = joblib.load("mnist_knn.joblib")

        self.guide = QLabel("검은색 숫자를 여기에 그려주세요 (0~9)", self)
        self.guide.move(20, 10)
        self.guide.resize(240, 20)

        self.okBtn = QPushButton("예측 및 적용", self)
        self.okBtn.move(90, 245)
        self.okBtn.clicked.connect(self.predictDigit)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = True
            self.lastPoint = event.pos()

    def mouseMoveEvent(self, event):
        if (event.buttons() & Qt.LeftButton) and self.drawing:
            painter = QPainter(self.image)
            pen = QPen(Qt.black, 15, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin)
            painter.setPen(pen)
            painter.drawLine(self.lastPoint, event.pos())
            self.lastPoint = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = False

    def paintEvent(self, event):
        canvasPainter = QPainter(self)
        canvasPainter.fillRect(self.rect(), Qt.white)
        canvasPainter.drawImage(self.rect(), self.image, self.image.rect())

    def predictDigit(self):
        img = self.image.copy().scaled(28, 28).convertToFormat(QImage.Format_Grayscale8)
        ptr = img.bits()
        ptr.setsize(img.byteCount())
        arr = np.array(ptr).reshape(1, 28 * 28).astype("float32") / 255.0
        digit = self.model.predict(arr)[0]
        self.parent().setRotationFromDigit(int(digit))
        self.accept()

class AutoImageEnhancer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("필터 선택 이미지 보정기")
        self.image = None
        self.original = None
        self.applied_filters = {}  # 필터 이름: 강도
        self.initUI()

    def initUI(self):
        openBtn = QPushButton("이미지 열기")
        saveBtn = QPushButton("이미지 저장")
        resetBtn = QPushButton("초기화")
        applyBtn = QPushButton("필터 추가")
        clearBtn = QPushButton("필터 모두 제거")
        compareBtn = QPushButton("Before/After 보기")
        digitRotateBtn = QPushButton("숫자 쓰기 (회전)")

        self.comboBox = QComboBox()
        self.comboBox.addItems([
            "CLAHE (대비 보정)",
            "블러",
            "샤프닝",
            "에지 검출"
        ])
        self.comboBox.currentIndexChanged.connect(self.resetSlider)

        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(1)
        self.slider.setMaximum(10)
        self.slider.setValue(1)
        self.slider.setTickInterval(1)
        self.slider.setTickPosition(QSlider.TicksBelow)

        openBtn.clicked.connect(self.openImage)
        saveBtn.clicked.connect(self.saveImage)
        resetBtn.clicked.connect(self.resetImage)
        applyBtn.clicked.connect(self.addFilter)
        clearBtn.clicked.connect(self.clearFilters)
        compareBtn.clicked.connect(self.showBeforeAfter)
        digitRotateBtn.clicked.connect(self.openDigitDialog)

        self.imageLabel = QLabel("이미지를 열어 주세요")
        self.imageLabel.setAlignment(Qt.AlignCenter)
        self.imageLabel.setFixedSize(512, 512)

        layout = QVBoxLayout()
        hlayout1 = QHBoxLayout()
        hlayout1.addWidget(self.comboBox)
        hlayout1.addWidget(applyBtn)
        hlayout1.addWidget(clearBtn)
        hlayout2 = QHBoxLayout()
        hlayout2.addWidget(QLabel("강도 조절"))
        hlayout2.addWidget(self.slider)

        layout.addWidget(openBtn)
        layout.addWidget(saveBtn)
        layout.addWidget(resetBtn)
        layout.addLayout(hlayout1)
        layout.addLayout(hlayout2)
        layout.addWidget(digitRotateBtn)
        layout.addWidget(compareBtn)
        layout.addWidget(self.imageLabel)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def resetSlider(self):
        self.slider.setValue(1)

    def openImage(self):
        path, _ = QFileDialog.getOpenFileName(self, "이미지 열기", "", "Image Files (*.png *.jpg *.jpeg *.bmp)")
        if path and os.path.exists(path):
            img = cv2.imread(path)
            if img is not None:
                self.original = img.copy()
                self.image = img.copy()
                self.applied_filters.clear()
                self.displayImage()

    def saveImage(self):
        if self.image is not None:
            path, _ = QFileDialog.getSaveFileName(self, "이미지 저장", "", "PNG Files (*.png);;JPEG Files (*.jpg)")
            if path:
                cv2.imwrite(path, self.image)

    def resetImage(self):
        if self.original is not None:
            self.image = self.original.copy()
            self.applied_filters.clear()
            self.displayImage()

    def clearFilters(self):
        self.applied_filters.clear()
        self.applyAllFilters()

    def addFilter(self):
        if self.original is None:
            return
        filter_name = self.comboBox.currentText()
        level = self.slider.value()
        self.applied_filters[filter_name] = level  # 중복되지 않게 덮어씀
        self.applyAllFilters()

    def applyAllFilters(self):
        img = self.original.copy()
        for f_name, level in self.applied_filters.items():
            if f_name == "CLAHE (대비 보정)":
                ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
                y, cr, cb = cv2.split(ycrcb)
                clahe = cv2.createCLAHE(clipLimit=level, tileGridSize=(8, 8))
                y = clahe.apply(y)
                ycrcb = cv2.merge([y, cr, cb])
                img = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

            elif f_name == "블러":
                ksize = level * 2 + 1
                img = cv2.GaussianBlur(img, (ksize, ksize), 0)

            elif f_name == "샤프닝":
                kernel = np.array([[0, -1, 0],
                                   [-1, 5 + level, -1],
                                   [0, -1, 0]])
                img = cv2.filter2D(img, -1, kernel)

            elif f_name == "에지 검출":
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                edge = cv2.Canny(gray, 50 * level, 150 * level)
                img = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

        self.image = img.copy()
        self.displayImage()

    def setRotationFromDigit(self, degree):
        if self.image is not None:
            h, w = self.image.shape[:2]
            M = cv2.getRotationMatrix2D((w / 2, h / 2), degree * 10, 1)
            self.image = cv2.warpAffine(self.image, M, (w, h))
            self.displayImage()

    def openDigitDialog(self):
        if self.image is not None:
            dialog = DrawDigitDialog(self)
            dialog.exec_()

    def showBeforeAfter(self):
        if self.original is not None and self.image is not None:
            dialog = BeforeAfterDialog(self.original, self.image)
            dialog.exec_()

    def displayImage(self):
        img = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        h, w, ch = img.shape
        bytesPerLine = ch * w
        qimg = QImage(img.data, w, h, bytesPerLine, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimg).scaled(512, 512, Qt.KeepAspectRatio)
        self.imageLabel.setPixmap(pixmap)
        self.imageLabel.repaint()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = AutoImageEnhancer()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Python\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
